## Reconstructing images from few pixels

In [ ]:
from picIHT import *
import scipy.ndimage as spimg
import glob
import os
import myprofile

### -----------------------------------------------
### ---------- set image folder -------------------
folder = "/Users/Martin/Pictures/Camera Roll/"
### -----------------------------------------------


# all jpg files in folder
list_camera_pics = glob.glob(folder+'*.jpg') 
# file name of latest one
latest_pic = max(list_camera_pics, key=os.path.getctime) 
#latest_pic = "/Users/Martin/Pictures/Camera Roll/DSCN0926.JPG"

#read image
Xorig = spimg.imread(latest_pic,flatten=True, mode='L')

# decrease resolution to increase speed
relative_resolution = 1
X0 = spimg.zoom(Xorig, relative_resolution)

# picture size
shape = X0.shape
n = np.prod(shape)

# choose basis
#L = 3
#amp = np.linspace(1,1,L)
#amp = np.kron(amp, np.ones(3) )
#amp = np.insert(amp,0, 10 ) #prepend (position 0)
                
T = WT(shape, wavelet='db6',level=3, amplify=10)

#T = DCT(shape)

#check compressibility
# estimated compressibility

## hard TO
s = int( np.prod(shape)/20 )
TO = hardTO(s)
## soft TO
tau = 10
#TO = softTO(tau)

cX0 = compress(T,TO,X0)

#X0=cX0
#pltPic(X0)

In [ ]:
# nr. of pixels to keep
m = int(.20*n)

# generate subsampled picture
mask = getRandMask(n, m)
Xsub = np.zeros(shape)
Xsub.flat[mask] = X0.flat[mask]
pltPic(Xsub)

In [ ]:
sparsity_guess = int(.8*s)
tau = 15
#thOp = hardTO(sparsity_guess)
thOp = softTO(tau)

#with myprofile.Profiler(fname='profile.dat'):
Xrec=estimate(T, thOp, mask, Xsub, n_steps=150, X0=X0)

pltPic(Xrec)

In [ ]:
pltPic(X0)

In [ ]:
#tbd: add animation

#first exercise
import time
plt.ion()
x = np.linspace(-1,1,51)
plt.plot(np.sin(x))
for i in range(10):
    plt.plot([np.sin(i+j) for j in x], animated=True)
    # make it appear immediately
    plt.draw()
    time.sleep(1)

# Reconstructing a random image

In [ ]:
# Create random picture
shape = (600,800)
T = DCT(shape)
s = 20

X0 = randomPic(T,s)
pltPic(X0)

n = np.prod(shape)
m = int(n/20)

mask = getRandMask(n, m)
Xsub = np.zeros(shape)
#measurement vector:
Xsub.flat[mask] = X0.flat[mask]
pltPic(Xsub)

Xrec=estimate(T,s,mask,Xsub,n_steps=100, X0=X0)

pltPic(Xrec)